

# Taller: Análisis de Hipótesis con Datos de la ENS 2016-2017



## 1. Introducción



En investigación cuantitativa, la docimacia de hipótesis (o prueba de hipótesis) permite evaluar si las diferencias observadas en los datos pueden atribuirse al azar o reflejan patrones reales en la población. Este taller tiene como objetivo introducir herramientas estadísticas para comprobar hipótesis en datos reales, utilizando Python como herramienta de análisis.



A partir de la Encuesta Nacional de Salud (ENS) 2016-2017, trabajaremos con variables demográficas, biométricas y clínicas para:

- Verificar supuestos de normalidad.

- Comparar variables entre grupos (por ejemplo, hombres vs. mujeres).

- Analizar diferencias en variables continuas y categóricas.



Este análisis es clave para evaluar desigualdades en salud, diseñar políticas basadas en evidencia y generar conclusiones confiables a partir de datos poblacionales.



## 2. Objetivos

Al finalizar este taller, los y las estudiantes serán capaces de:



* Evaluar la normalidad de variables cuantitativas

- Verificar normalidad en variables como Edad, Peso, Talla, Glicemia y Colesterol.

* Comparar variables continuas entre grupos

- Aplicar pruebas t de Student para muestras independientes (hombres vs. mujeres).

- Comparar variables como Peso, IMC, y Glicemia por sexo.

* Comparar variables categóricas u ordinales

- Evaluar diferencias en número de comorbilidades o nivel educacional entre grupos.

- Utilizar pruebas como chi-cuadrado o pruebas no paramétricas.

* Interpretar resultados

- Comprender el significado de los valores p.

- Formular y contrastar hipótesis nula y alternativa.

- Traducir los resultados estadísticos en hallazgos interpretables en contexto de salud pública.



## 3. Dataset



La Encuesta Nacional de Salud (ENS) 2016-2017 es un estudio representativo de la población chilena que recopila datos sobre salud física y mental, factores de riesgo, estilo de vida y uso de servicios sanitarios. Esta información permite caracterizar el estado de salud y sus determinantes a nivel nacional.En este taller trabajaremos con un subconjunto de variables relevantes para el análisis estadístico de hipótesis. Los datos son de acceso público y están disponibles en este repositorio. A través de este conjunto de datos, aprenderemos a aplicar pruebas estadísticas clásicas sobre variables reales, extrayendo conclusiones aplicables a la investigación en salud pública. Los datos se encuentran [disponibles públicamente](https://data.mendeley.com/datasets/3myh3nzcy3/1).

![img/ens.png]()


### 3.1 Librerías necesarias
Las siguientes librerias son necesarias para el análisis de datos y visualización. 

In [1]:
%%capture
# Instalar librerias para leer archivos SPSS
!pip install pyreadstat

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

### 3.2 Cargar Datos

Tal como se realizo en el taller anterior, se descargara la base de datos utilizando `wget` y luego se cargara la base de datos en la variable `ens` utilizando la función de pandas `read_spss`.  

In [5]:
# Descargar ENS
!wget https://github.com/iHealthInstitute/Talleres_Diplomado_iHealth/raw/refs/heads/main/data/ENS2016-2017.sav

--2025-07-15 16:47:22--  https://github.com/iHealthInstitute/Talleres_Diplomado_iHealth/raw/refs/heads/main/data/ENS2016-2017.sav
Resolviendo github.com (github.com)... 20.201.28.151
Conectando con github.com (github.com)[20.201.28.151]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: https://raw.githubusercontent.com/iHealthInstitute/Talleres_Diplomado_iHealth/refs/heads/main/data/ENS2016-2017.sav [siguiendo]
--2025-07-15 16:47:23--  https://raw.githubusercontent.com/iHealthInstitute/Talleres_Diplomado_iHealth/refs/heads/main/data/ENS2016-2017.sav
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[185.199.110.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 34441286 (33M) [application/octet-stream]
Grabando a: «ENS2016-2017.sav.3»

ENS2016-2017.sav.3  100%[==============

In [7]:
ens = pd.read_spss('ENS2016-2017.sav')
print(ens.head())
print(ens.shape)

   IdEncuesta  FechaInicioF1               Region    Comuna    Zona  \
0     20006.0   1.369894e+10  XIII. Metropolitana  Santiago  URBANA   
1     20008.0   1.369964e+10  XIII. Metropolitana  Santiago  URBANA   
2     20011.0   1.369955e+10  XIII. Metropolitana  Santiago  URBANA   
3     20012.0   1.369903e+10  XIII. Metropolitana  Santiago  URBANA   
4     20013.0   1.369902e+10  XIII. Metropolitana  Santiago  URBANA   

   IdSegmento  IdPersona_1      Ident7  Edad Edad_Codificada  ...  \
0  13101101.0        241.0  1977-12-18  38.0         25 - 44  ...   
1  13101101.0        197.0  1991-10-23  25.0         25 - 44  ...   
2  13101102.0        321.0  1996-05-31  20.0           15-24  ...   
3  13101102.0        245.0  1931-04-14  85.0             65+  ...   
4  13101102.0        242.0  1975-06-24  41.0         25 - 44  ...   

  fg_CKDschwartz_diminuido_60 fg_CKDschwartz_diminuido_30 Fechaini_F1  \
0                         NaN                         NaN  2016-11-19   
1           